# Use RASA DSL to Configure DeepPavlov's GO-Bot

At DeepPavlov, we support a variety of industry-wide and popular standards to support developing Conversational AI solutions.
DSLs, known as Domain-Specific Languages, provide a rich mechanism to define the behavior, or "the what", while,
the underlying system uses the parser to transform these definitions into commands that implement this behavior, or "the how"
using the system's components.
Until very recently we supported two such DSLs, including industry-standard <a href="http://docs.deeppavlov.ai/en/master/features/skills/aiml_skill.html">AIML</a>, as well as <a href="http://docs.deeppavlov.ai/en/master/features/skills/dsl_skill.html">DSL</a> designed by one our partners, EORA.

In this tutorial, you will learn how to use another industrial DSL, or, better said, set of DSLs, introduced by RASA.ai,
to build simple goal-oriented chatbots using DeepPavlov's GO-bot.

As discussed in our [blog post](todo/link), this is the very beginning of our work focused on supporting RASA DSLs as a way to configure DeepPavlov-based
goal-oriented chatbots, and therefore 

To configure a DeepPavlov-based goal-oriented chatbot using these DSLs, you need to have at least three basic config files:
* `stories.md` (or `stories-{trn, tst, val}.md` but these are just subsamples)
* `nlu.md`
* `domain.yml`

These files allow you to define 3 key elements of the chatbot, including product-level stories, NLU training data, and your chatbot's domain.

## Concepts Behind Stories.md, NLU.md, and Domain.yml

### `stories.md`



`stories.md` is a mechanism used to teach your chatbot how to respond to user messages. It allows you to control your chatbot's dialog management.

These "stories" model real conversations between a user and a chatbot. This Markdown-based file is used to define a list of
*stories*, and each *story* can have a list of one or more *intents* with (optional) corresponding *slots*, where each *intent*
has one or more corresponding actions taken by the chatbot.

These actions, in general, can be anything, from simple message replies, to programmable actions that call APIs of other services.
*Note:* In this version, supported actions are limited to simple message replies.

In a way, it can be seen as a *dialogues dataset*.

#### format

Stories file is a markdown file of the following format:

```markdown
## story_title(not used by algorithm, but useful to work with for humans)
* user_action_label{"1st_slot_present_in_action": "slot1_value", .., "Nth_slot_present_in_action": "slotN_value"}
 - system_respective_utterance
* another_user_action_of_the_same_format
  - another_system_response
...

## another_story_title
...

```

**See examples below in this tutorial**

### `nlu.md`


`nlu.md` represents an NLU model of your chatbot. It allows you to provide training examples that show how your chatbot should
understand user messages, and then train a model through these examples.

While DeepPavlov's GO-bot supports JSON-based DSTC-2 format for training data, this Markdown Format introduced by RASA is the easiest one for humans to read and write.

#### format

NLU file is a markdown file of the following format:

```markdown
## intent:possible_user_action_label_1
- An example of user text that has the possible_user_action_label_1 action label
- Another example of user text that has the possible_user_action_label_1 action label
...

## intent:possible_user_action_label_N
- An example of user text that has the (possible_user_action_label_N)[action_label] action label
<!-- Slotfilling dataset is provided as an inline markup of user texts -->
...

```

**See examples below in this tutorial**

### `domain.yml`

`domain.yml` helps you to define the universe your chatbot lives in: what user inputs it expects to get, what actions it should be able to predict,
how to respond, and what information to store.
This YML format is relatively simple, and it can be seen as a dictionary of all components of your chatbot, including but not limited to intents,
actions, responses, and other things.

#### format

Domain file is a YAML file of the following format:

```yaml
# slots section lists the possible slot names (aka slot types) 
# that are used in the domain (i.e. relevant for bot's tasks)
# currently only type: text is supported
slots:
  slot1_name:
    type: text
  ...
  slotN_name:
    type: text

# entities list now follows the slots list 2nd level keys 
# and is present to support upcoming features. Stay tuned for updates with this!
entities:
- slot1_name
...
- slotN_name

# intents section lists the intents that can appear in the stories
# being kept together they do describe the user-side part of go-bot's experience
intents:
  - user_action_label
  - another_user_action_of_the_same_format
  ...

# responses section lists the system response templates.
# Despite system response' titles being usually informative themselves
#   (one could even find them more appropriate when no actual "Natural Language" is needed 
#    (e.g. for buttons actions in bot apps))
# It is though extremely useful to be able to serialize the response title to text. 
# That's what this section content is needed for.
responses:
  system_utterance_1:
    - text: "The text that system responds with"
  another_system_response:
    - text: "Here some text again"

```

**See examples below in this tutorial**

## Basic Chatbot

Let's build the simplest chatbot possible.
This chatbot will be capable of processing three intents: *greeting*, *goodbye*, and *thanks*.

In [1]:
DP_MIN_DEMO_DIR = "dp_minimal_demo_dir"  # we will work in this folder

In [2]:
import os

%cd /content
os.makedirs(DP_MIN_DEMO_DIR, exist_ok=True)
%cd {DP_MIN_DEMO_DIR}

/content
/content/dp_minimal_demo_dir


`stories.md` here is pretty straightforward. In it, you define 3 stories, each having its own intent and reply (utterance).
Take into account the fact that you can combine all of these intents under one story, or add two intents to one stories, and third to another one.

In [3]:
%%writefile stories.md

## greet
* greet
  - utter_greet

## thank
* thank
  - utter_noworries

## goodbye
* bye
  - utter_bye

Writing stories.md


`nlu.md` has an NLU training data that enables DeepPavlov to recognize user phrases as belonging to one the pre-defined intents.

In [4]:
%%writefile nlu.md

## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks

## intent:bye
- goodbye
- goodnight
- good bye
- good night
- see ya
- toodle-oo
- bye bye
- gotta go
- farewell

## intent:thank
- Thanks
- Thank you
- Thank you so much
- Thanks bot
- Thanks for that
- cheers


Writing nlu.md


`domain.yml` lists the list of: 
* possible user action intents 
* possible system response actions

*Note:* Entities and slots are omitted in this example. See the more sophisticated example below to see how they are defined in the `domain.yml`.

In [5]:
%%writefile domain.yml

intents:
  - greet
  - bye
  - thank

responses:
  utter_noworries:
  - text: No worries!
  utter_greet:
  - text: Hi
  utter_bye:
  - text: Bye!

Writing domain.yml


The next step is to install the `deeppavlov` library.

In [6]:
!pip install git+https://github.com/deepmipt/DeepPavlov.git@feature/gobot-md-yaml-config
!python -m deeppavlov install gobot_simple_dstc2

  Cloning https://github.com/deepmipt/DeepPavlov.git (to revision feature/gobot-md-yaml-config) to /tmp/pip-req-build-ejpo32hx
  Running command git clone -q https://github.com/deepmipt/DeepPavlov.git /tmp/pip-req-build-ejpo32hx
  Running command git checkout -b feature/gobot-md-yaml-config --track origin/feature/gobot-md-yaml-config
  Switched to a new branch 'feature/gobot-md-yaml-config'
  Branch 'feature/gobot-md-yaml-config' set up to track remote branch 'feature/gobot-md-yaml-config' from 'origin'.
     |████████████████████████████████| 51kB 3.6MB/s 
     |████████████████████████████████| 2.1MB 12.4MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 1.5MB 38.7MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 10.4MB 46.2MB/s 
     |████████████████████████████████| 512kB 42.6MB/s 
     |████████████████████████████████| 7.3MB 28.7MB/s 
     |████████████████████████████████| 51kB 

2020-08-03 14:27:36.589 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_simple_dstc2' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_simple_dstc2.json'
     |████████████████████████████████| 696kB 4.5MB/s 
     |████████████████████████████████| 110.5MB 94kB/s 
     |████████████████████████████████| 3.8MB 44.0MB/s 
     |████████████████████████████████| 512kB 34.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=0bd5e2b823c554a94069ed66595b9e6096e6e28d7e8945f65b77db1530d81165
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.

Define the path to our DSL-based configuration files above (the folder we are in right now) and the folder used to store the trained bot.

In [7]:

from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)

gobot_config['metadata']['variables']['DATA_PATH'] = '.'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

Since our data is basically the mock tutorial data we will use the same subsamples for all of train, test and valid subsamples. 

However, for a real system you should use different train, test, and valid sample stories.md files.

In [8]:
!cp stories.md stories-trn.md
!cp stories.md stories-tst.md 
!cp stories.md stories-val.md 

The next step is to train the bot:

In [9]:
from deeppavlov import train_model

train_model(gobot_config, download=True)

2020-08-03 14:28:50.38 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/datasets/gobot_md_yaml_minimal.tar.gz to /content/gobot_md_yaml_minimal.tar.gz
100%|██████████| 528/528 [00:00<00:00, 1.19MB/s]
2020-08-03 14:28:50.402 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /content/gobot_md_yaml_minimal.tar.gz archive into /content/dp_minimal_demo_dir
2020-08-03 14:28:50.770 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt
347MB [01:31, 3.79MB/s]
2020-08-03 14:30:22.758 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=False, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge obje

2020-08-03 14:30:26.700 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f9151db5f98>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slots': []}, {'text': 'toodle-oo', 'slots': []}, {'text': 'bye bye', 'slots': []}, {'text': 'gotta go', 'slots': []}, {'text': 'farewell', 'slots': []}]}, 'thank': {(





Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-08-03 14:31:05.99 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-08-03 14:31:05.187 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best per_item_action_accuracy of 0.3333


{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 0.3333}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

2020-08-03 14:31:45.101 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-08-03 14:31:45.107 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f914a3678d0>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slot

{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}


2020-08-03 14:31:45.113 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 306: AFTER MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f914a3678d0>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slots': []}, {'text': 'toodle-oo', 'slots': []}, {'text': 'bye bye', 'slots': []}, {'text': 'gotta go', 'slots': []}, {'text': 'farewell', 'slots': []}]}, 'thank': {()

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


Chainer[<deeppavlov.models.go_bot.wrapper.DialogComponentWrapper at 0x7f916c473e10>,

Finally, it's time to build our bot and experiment with it:

In [10]:
from deeppavlov import build_model
bot = build_model(gobot_config)

2020-08-03 14:32:23.253 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-08-03 14:32:23.258 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f9140ee9748>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slot

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


In [11]:
bot.reset()

bot(["start"])
bot(["Hi"])[0][0].actions_tuple

('utter_greet',)

Our bot answers with "greeting" to our "greeting". What will it respond to some grateful message?

In [12]:
bot.reset()
bot(["start"])

bot(["Thanks!"])[0][0].actions_tuple

('utter_noworries',)

Ok, "no woories" is an expected response. Let's check if the "goodbye" user message is processed with the corresponding reply:

In [13]:
bot.reset()
bot(["start"])

bot_response_actions = bot(["bye"])[0][0].actions_tuple

In [14]:
import yaml

system_utter2text = yaml.load(open("domain.yml"))["responses"]
system_utter2text[bot_response_actions[0]][0]["text"]

'Bye!'

## More Realistic Demo: Using the DSTSC Schema-Guided Dialogue Dataset

In [15]:
DP_BIG_DEMO_DIR = "dp_big_demo_dir"  # we'll work in this directory

In [16]:
import os

%cd /content
os.makedirs(DP_BIG_DEMO_DIR, exist_ok=True)
%cd {DP_BIG_DEMO_DIR}

/content
/content/dp_big_demo_dir


Now let's take the [Schema](https://github.com/google-research-datasets/dstc8-schema-guided-dialogue) dataset.

This dataset is consisted of lots of annotated human-machine conversations 
crowdsourced in an [M2M manner](https://arxiv.org/pdf/1801.04871.pdf) for various real-life scenarios and domains.

One of such domains is a *Restaurants* domain. 
Here, users are searhing for restaurants or booking tables via interacting with the bot.  
We will proceed with training a restaurants-assistant bot using the Restaurants domain data provided in the Schema dataset.



##### Download the data used in this tutorial section

In [17]:
# let's get the mentioned converted Schema-dataset subset
!wget http://files.deeppavlov.ai/datasets/schema_resto_md_yaml.tar.gz
!tar -zxf schema_resto_md_yaml.tar.gz 

--2020-08-03 14:33:02--  http://files.deeppavlov.ai/datasets/schema_resto_md_yaml.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98725 (96K) [application/octet-stream]
Saving to: ‘schema_resto_md_yaml.tar.gz’

schema_resto_md_yam 100%[===================>]  96.41K   274KB/s    in 0.4s    

2020-08-03 14:33:03 (274 KB/s) - ‘schema_resto_md_yaml.tar.gz’ saved [98725/98725]



##### Technical note on the origin of DSL-files used to train the restaurants bot.

Schema dataset is provided in DSTC (dialogue state tracking challenge) challenge format.
The DSTC format has its own advantages: it is very detailed and allows for various additional info to be incorporated to the dataset itsef.

But the DSL we're illustrating here is different from the DSTC: it is meant to be neat and minimalistic.

So we did perform an automatical conversion of the Schema Restaurants dataset from the DSTC format to RASA DSL.

### The role of additional NLU components in the workflow of building the Goal-Oriented Bot

When building the go-bot with deeppavlov one usually has to set up some NLU components.
All of these separate NLU components are optional by design in the go-bot pipeline except the slotfiller.

The goal of slotfiller is to recognize and normalize slot-filling info provided in user's utterances. 
That is, if user is informing the system with an explicit interest in booking a table *in London*, 
it is the goal of the slotfiller component to understand that the `city` slot information was explicitely provided. 

We're providing the pretrained slotfiller for the dataset. See [the small notebook](todo/link) on how the slotfiller was trained.

#### [TODO УБРАТЬ СЕКЦИЮ!!!] Training an additional slotfiller component for goal-oriented bot

In [18]:
from deeppavlov import configs, train_model
from deeppavlov.core.common.file import read_json

ner_config = read_json(configs.ner.ner_conll2003)
ner_config["dataset_reader"] = read_json(configs.ner.ner_few_shot_ru)["dataset_reader"]
ner_config["dataset_reader"]["data_path"] = "schema_resto_md_yaml/ner"

In [19]:
!python -m deeppavlov download ner_conll2003
!rm -r /root/.deeppavlov/models/ner_conll2003  # remove cached models if present

2020-08-03 14:33:06.630 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ner_conll2003' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/ner/ner_conll2003.json'
2020-08-03 14:33:08.205 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt?config=ner_conll2003 download because of matching hashes
2020-08-03 14:33:08.745 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/ner_conll2003_v5.tar.gz?config=ner_conll2003 to /root/.deeppavlov/ner_conll2003_v5.tar.gz
100% 3.12M/3.12M [00:01<00:00, 2.00MB/s]
2020-08-03 14:33:10.682 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/ner_conll2003_v5.tar.gz archive into /root/.deeppavlov/models


In [20]:
# if rm says that these files are missing that's ok 
#   (we just deleted the whole folder containing these paths ^__^)
# but if they're present we should delete them 
# cause they're checkpoints of some other model
!rm /root/.deeppavlov/models/ner_conll2003/{checkpoint,model_no_pos}*

rm: cannot remove '/root/.deeppavlov/models/ner_conll2003/checkpoint*': No such file or directory
rm: cannot remove '/root/.deeppavlov/models/ner_conll2003/model_no_pos*': No such file or directory


In [21]:
ner_model = train_model(ner_config)

2020-08-03 14:33:14.965 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-08-03 14:33:14.973 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-08-03 14:33:14.993 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 101: [saving vocabulary to /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-08-03 14:33:14.997 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]



Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use `tf.cast` instead.



2020-08-03 14:33:52.274 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-08-03 14:33:52.375 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-08-03 14:33:55.33 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 397: processed 1424 tokens with 32 phrases; found: 1049 phrases; correct: 0.

precision:  0.00%; recall:  0.00%; FB1:  0.00

	city: precision:  0.00%; recall:  0.00%; F1:  0.00 85

	cuisine: precision:  0.00%; recall:  0.00%; F1:  0.00 227

	date: precision:  0.00%; recall:  0.00%; F1:  0.00 248

	price_range: precision:  0.00%; recall:  0.00%; F1:  0.00 127

	restaurant_name: precision:  0.00%; recall:  0.00%; F1:  0.00 34

	time: precision:  0.00%; recall:  0.00%; F1:  0.00 328


2020-08-03 14:33:55.37 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best ner_f1 of 0


...

{"valid": {"eval_examples_count": 214, "metrics": {"ner_f1": 86.1538, "ner_token_f1": 85.4167}, "time_spent": "0:00:01"}}


2020-08-03 14:36:03.169 DEBUG in 'deeppavlov.metrics.fmeasure'['fmeasure'] at line 397: processed 1846 tokens with 50 phrases; found: 54 phrases; correct: 0.

precision:  75.93%; recall:  82.00%; FB1:  78.85

	city: precision:  73.33%; recall:  84.62%; F1:  78.57 15

	cuisine: precision:  86.67%; recall:  92.86%; F1:  89.66 15

	date: precision:  50.00%; recall:  33.33%; F1:  40.00 4

	price_range: precision:  75.00%; recall:  100.00%; F1:  85.71 4

	restaurant_name: precision:  0.00%; recall:  0.00%; F1:  0.00 1

	time: precision:  80.00%; recall:  92.31%; F1:  85.71 15




{"test": {"eval_examples_count": 267, "metrics": {"ner_f1": 78.8462, "ner_token_f1": 88.1188}, "time_spent": "0:00:01"}}


2020-08-03 14:36:03.459 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-08-03 14:36:03.465 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-08-03 14:36:03.467 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-08-03 14:36:03.470 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-08-03 14:36:40.652 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTM

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


Ner performs OK for everything except the restaurants' name identification.  
That's mostly because we avoid the usage of restaurants database here.

Let's check the NER performance running some slotfilling experiment

In [22]:
import json

json.dump(ner_config, open('ner_config.json', 'wt'))

In [23]:
import json
from deeppavlov import configs

from deeppavlov.core.common.file import read_json
from deeppavlov import build_model

slotfill_config = read_json(configs.ner.slotfill_dstc2)
slotfill_config['dataset_reader']['class_name'] = "md_yaml_dialogs_reader"
slotfill_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
slotfill_config['metadata']['variables']['SLOT_VALS_PATH'] = 'schema_resto_md_yaml/slotfill.json'
slotfill_config["chainer"]["pipe"][-1]["load_path"] = "schema_resto_md_yaml/slotfill.json"
slotfill_config["metadata"]["variables"]["NER_CONFIG_PATH"] = "ner_config.json"
slotfiller = build_model(slotfill_config, download=False)

2020-08-03 14:36:42.862 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/word.dict]
2020-08-03 14:36:42.866 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/tag.dict]
2020-08-03 14:36:42.869 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_conll2003/char.dict]
2020-08-03 14:36:42.871 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-08-03 14:37:20.16 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMC

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-08-03 14:37:21.905 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.


In [24]:
slotfiller(["i'm looking for a thai food somewhere in SFO"])

[{'city': 'SFO', 'cuisine': 'Thai'}]

Seems OK. Let's save our slotfiller config to train and evaluate the restaurants bot, finally.

In [25]:
import json

json.dump(slotfill_config, open('slotfill_config.json', 'wt'))

### The restaurants assistant bot

Again, we will use the `stories.md`, `nlu.md` and `domain.yml` to train the bot.

But the data now is a bit more sophisticated to mimic the real scenario of the go-bot usage.

The only strong simplification we perform here is that we excluded all the interaction with the DB from the training data.  
This would damage the overall performance of the bot though it should still be able to generalize and use the global patterns underlying in conversations. 

Let's first take a look at our DSL files

`stories.md`

Stories are still showing the metainformation beyond the possible dialogues.
That is user intents are provided in the `intent_name+informed_slots_info` format. 
And the system respones are provided as response-class labels too.

In [26]:
!head -500 schema_resto_md_yaml/stories.md | tail -30

  - utter_INFORM_Cuisine+INFORM_StreetAddress+NOTIFY_SUCCESS
* THANK_YOU+GOODBYE{}
  - utter_GOODBYE

## 25
* hi{}
  - utter_hi
* INFORM_INTENT_Intent{"intent": "FindRestaurants"}
  - utter_REQUEST_City
* INFORM_City{"city": "Oakland"}
  - utter_REQUEST_Cuisine
* INFORM_Cuisine{"cuisine": "Fish"}
  - utter_OFFER_RestaurantName+OFFER_City+INFORM_COUNT_Count
* REQUEST_ALTS{}
  - utter_OFFER_RestaurantName+OFFER_City
* INFORM_PriceRange+REQUEST_ALTS{"price_range": "moderate"}
  - utter_OFFER_RestaurantName+OFFER_City+INFORM_COUNT_Count
* REQUEST_ServesAlcohol+REQUEST_HasLiveMusic{}
  - utter_INFORM_HasLiveMusic+INFORM_ServesAlcohol
* INFORM_INTENT_Intent+SELECT{"intent": "ReserveRestaurant"}
  - utter_REQUEST_Time
* INFORM_PartySize+INFORM_Time{"party_size": "2", "time": "10:30 in the morning"}
  - utter_CONFIRM_RestaurantName+CONFIRM_City+CONFIRM_Time+CONFIRM_PartySize+CONFIRM_Date
* REQUEST_StreetAddress+REQUEST_PhoneNumber+AFFIRM{}
  - utter_NOTIFY_FAILURE+REQ_MORE
* INFORM_PartySize+I

`nlu.md`

NLU file still shows the examples of user utterances for the possible intent classes.
The slotfilling and NER information is provided in the form of inline mark-up.

In [27]:
!head -50 schema_resto_md_yaml/nlu.md | tail -20

- [Evening 7:15](time) at Mcdonald's.
- [6:15 in the evening](time) at Bistro liaison.
- Let's set it up for [6:30 pm](time) at Sakoon.
- I would like a reservation in [Flames Eatery](restaurant_name) at [19:00](time)
- I'm thinking about Rice Bowl, at [morning 11:30](time)

## intent:REQUEST_Cuisine+AFFIRM
- Sounds great. Do you know if they cook [Taiwanese](cuisine) food?
- yeah. what do they serve?

## intent:AFFIRM
- Yes
- That's great.
- Sure that sounds good.
- That works!
- that would be great.
- That sounds good.
- Yes that works for me
- Sure, that sounds great!
- This is good for me


`domain.yml`
The domain file now provides the list of slots and entities relevant for the bot as well as possible intent classes and system response text templates.

In [28]:
!head -40 schema_resto_md_yaml/domain.yml && echo "..."
!grep -B 1 -A 10 responses schema_resto_md_yaml/domain.yml && echo "..."

slots:
  price_range:
    type: text
  cuisine:
    type: text
  date:
    type: text
  restaurant_name:
    type: text
  intent:
    type: text
  party_size:
    type: text
  time:
    type: text
  has_live_music:
    type: text
  city:
    type: text
  serves_alcohol:
    type: text

entities:
- price_range
- cuisine
- date
- restaurant_name
- intent
- party_size
- time
- has_live_music
- city
- serves_alcohol

intents:
  - INFORM_RestaurantName+INFORM_Time
  - THANK_YOU+GOODBYE
  - INFORM_Time+INFORM_RestaurantName
  - INFORM_PriceRange+REQUEST_ALTS
  - REQUEST_PhoneNumber+REQUEST_StreetAddress+AFFIRM
...

responses:
  utter_REQUEST_Cuisine:
    - text: "What are you in the mood for? {cuisine}? {cuisine}"
  utter_REQUEST_City:
    - text: "What city are you interested in?"
  utter_OFFER_RestaurantName+OFFER_City:
    - text: "Great! {restaurant_name} is a great restaurant in {city}"
  utter_INFORM_PhoneNumber+INFORM_HasLiveMusic:
    - text: "The contact number is {phone_number}"
  

These files are what we need to train the GO-bot so let's do it.

In [29]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)
gobot_config['chainer']['pipe'][-1]['slot_filler'] = {"config_path": "slotfill_config.json"}

gobot_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

Since our data is the tutorial data we will use the same subsamples for all of train, test and valid subsamples.

However, for a real system you should use different train, test, and valid sample stories.md files.

In [30]:
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-trn.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-tst.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-val.md

In [49]:
from deeppavlov import train_model

train_model(gobot_config, download=False);

2020-08-03 15:02:25.894 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-08-03 15:02:25.987 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-08-03 15:02:27.735 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_conll2003/model_no_pos]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-08-03 15:02:27.919 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-08-03 15:02:27.923 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-08-03 15:03:05.966 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-08-03 15:03:05.972 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [50]:
bot = build_model(gobot_config)

2020-08-03 15:04:06.723 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-08-03 15:04:06.812 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-08-03 15:04:08.538 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_conll2003/model_no_pos]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_conll2003/model_no_pos


2020-08-03 15:04:08.713 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/content/dp_big_demo_dir/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-08-03 15:04:08.716 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-08-03 15:04:45.989 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-08-03 15:04:45.996 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [51]:
bot.reset()

bot(["Hey!"])[0][0].actions_tuple

('utter_hi',)

Ok the bot performs OK on greeting

In [52]:
bot(["I'd like to find a restaurant for this evening"])[0][0].actions_tuple

('utter_REQUEST_City',)

The bot asks for an inevitably necessary information.

In [53]:
bot(["Somewhere in Oakland, sushi and for two people please"])[0][0].actions_tuple

('utter_OFFER_RestaurantName', 'OFFER_City')

And so on.

In [54]:
bot(["Cool! That's what I was looking for, thanks!"])[0][0].actions_tuple

('utter_REQUEST_Time',)

Let's say goodbye to our bot

In [55]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_CONFIRM_RestaurantName',
 'CONFIRM_City',
 'CONFIRM_Time',
 'CONFIRM_Date',
 'CONFIRM_PartySize')

After the goodbye bot does perform uninterpretable. That's what most of recurrent system do after the end of the sequence cause no one reaches that poing. 

In [56]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_NOTIFY_SUCCESS',)

## Final words

More of in-depth examples of goal-oriented bots and their features are coming with future releases, stay tuned!